In [56]:
import numpy as np
import pandas as pd
import torchvision.transforms as transforms
import time
import random
import os
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torch.optim.lr_scheduler as lr_scheduler
from torch.nn import functional as F
import torchvision
from torchvision import transforms, datasets, models, utils
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from skimage import io, transform
from skimage.transform import AffineTransform, warp
from tqdm import tqdm
import albumentations as A
from torch.cuda.amp import autocast, GradScaler
from glob import glob
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm_notebook
import csv

device = torch.device('cuda')

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def seeding(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [57]:
class VisualField(Dataset):
    def __init__(self,csv_file):
        self.annotations = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self,index):
        probability = []
        td = []
        for i in range(12, 66):
            if i != 37 and i != 46:
                td.append(self.annotations.iloc[index, i])
        for i in range(68, 122):
            if i != 93 and i != 102:
                probability.append(self.annotations.iloc[index, i])

        probability = np.array(probability)
        td = np.array(td)

        td = td[~np.isnan(td)]

        newtd = np.copy(td)

        td = torch.tensor(td)
        probability = torch.tensor(probability)

        return (torch.tensor(newtd),td,probability)

In [ ]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)

from google.colab import drive
drive.mount('/content/drive')

N = "Fast"

path = "drive/MyDrive/VisualField"

simulationData = pd.read_csv(f"{path}/{N}P.csv")

train_data = simulationData.groupby('simulation', group_keys=False).apply(lambda x: x.sample(frac=0.8, random_state=42))
val_data = simulationData.drop(train_data.index)

simulationData.to_csv(f"{path}/simulationData.csv", index=False)
train_data.to_csv(f"{path}/simulation_train.csv", index=False)
val_data.to_csv(f"{path}/simulation_val.csv", index=False)

dataset_sim = VisualField(csv_file=f"{path}/simulationData.csv")
dataset_train = VisualField(csv_file=f"{path}/simulation_train.csv")
dataset_val = VisualField(csv_file=f"{path}/simulation_val.csv")

batch_size =  64
data_loader = DataLoader(dataset_sim, batch_size=batch_size,shuffle = False)
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size,shuffle = False)

In [59]:
class VFAutoencoder(nn.Module):
    def __init__(self):
        super(VFAutoencoder, self).__init__()
        self.e1 = nn.Linear(104, 32)
        self.e2 = nn.Linear(32, 16)
        self.mu_layer = nn.Linear(16, 16)
        self.std_layer = nn.Linear(16, 16)
        self.d1 = nn.Linear(16, 32)
        self.d2 = nn.Linear(32, 52)
        self.relu = nn.ReLU()

        self.N = torch.distributions.Normal(0, 1)
        self.kl = 0

    def forward(self, TD, Probability):
        combined = torch.cat((TD, Probability), dim=1)
        encoded1 = self.relu(self.e1(combined))
        encoded2 = self.relu(self.e2(encoded1))

        mu = self.mu_layer(encoded2)
        logvar = self.std_layer(encoded2)
        sigma = torch.exp(0.5 * logvar)

        epsilon = torch.randn_like(sigma)
        z = mu + sigma * epsilon

        self.kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / TD.size(0)

        decoded1 = self.relu(self.d1(z))
        decoded2 = self.d2(decoded1)
        return decoded2

batchsize = 64
input_size = 52
TD = torch.randn(batchsize, input_size).cuda()
Probability = torch.randn(batchsize, input_size).cuda()
model = VFAutoencoder().cuda()
output = model(TD, Probability)

In [60]:
def train2(model, loader, optimizer, device):
    epoch_loss = 0.0
    model.train()
    with tqdm_notebook(total=len(loader), desc='Training', leave=False) as pbar:
        for (_,TD, Prob) in loader:
            TD = TD.to(device).float()
            Prob = Prob.to(device).float()

            optimizer.zero_grad()
            pred = model(TD, Prob)
            loss = F.mse_loss(pred, TD) + model.kl

            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            torch.cuda.empty_cache()

            pbar.set_postfix({'Loss': loss.item()})
            pbar.update(1)

    epoch_loss = epoch_loss / len(loader)

    return epoch_loss

def evaluate2(model, loader, optimizer, device):
    epoch_loss = 0.0
    model.eval()

    with torch.no_grad():
        for (_,TD,Prob) in loader:

            TD = TD.to(device).float()
            Prob = Prob.to(device).float()

            optimizer.zero_grad()
            pred = model(TD,Prob)
            loss= F.mse_loss(pred, TD) + model.kl

            epoch_loss += loss.item()
            torch.cuda.empty_cache()

        epoch_loss = epoch_loss/len(loader)
    return epoch_loss

In [ ]:
def TrainModel2(numE, lr, seed):

    device = torch.device('cuda')
    model = VFAutoencoder()

    model = model.to(device)

    num_epochs = numE
    learning_rate = lr
    checkpoint_path = f"{path}/VAE_P_modified.pth"

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
    lowest_loss = 1000000
    patience = 0

    for epoch in range(num_epochs):
        start_time = time.time()
        train_loss = train2(model, train_loader, optimizer, device)
        valid_loss = evaluate2(model, val_loader, optimizer, device)

        scheduler.step(valid_loss)

        if valid_loss < lowest_loss:
            data_str = f"Valid Loss decreased from {lowest_loss:2.4f} to {valid_loss:2.4f}. Saving checkpoint: {checkpoint_path}"
            torch.save(model.state_dict(), checkpoint_path)

        if epoch % 10 == 0:
            end_time = time.time()
            epoch_mins, epoch_secs = epoch_time(start_time, end_time)
            data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
            data_str += f'\tTrain Loss: {train_loss:.3f}\n'
            data_str += f'\t Val. Loss: {valid_loss:.3f}\n'
            print(data_str)

        patience = patience + 1
        if valid_loss < lowest_loss:
            lowest_loss = valid_loss
            patience = 0
        if patience == 10:
            break

In [ ]:
TrainModel2(1000, 0.001, 42)

In [ ]:
checkpoint_path = f"{path}/VAE_P_modified.pth"
predicted = []

def test(model, loader, optimizer, device):
    epoch_loss = 0.0
    model.eval()

    with torch.no_grad():
        for (_,TD,Prob) in loader:

            TD = TD.to(device).float()
            Prob = Prob.to(device).float()

            optimizer.zero_grad()
            pred = model(TD,Prob)
            for i in pred:
                predicted.append(i.detach().cpu().numpy())

            loss= F.mse_loss(pred, TD) + model.kl

            epoch_loss += loss.item()
            torch.cuda.empty_cache()

        epoch_loss = epoch_loss/len(loader)
    return epoch_loss


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VFAutoencoder().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()


test(model,data_loader,optimizer,device)

In [ ]:
def write_list_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

list_of_lists = predicted

csv_filename = f"{path}/{N}_Vp.csv"

write_list_to_csv(list_of_lists, csv_filename)